In [1]:
import pandas as pd

In [2]:
df=pd.read_csv(r'train_data_live.csv')

In [3]:
df

,URL,Abstract,Category
0,/[i9https://patents.google.com/patent/US904030...,\n Methods and compositions for targeted d...,Non-Alcohol
1,https://patents.google.com/patent/US9040109B2/en,\n The present invention provides stabiliz...,Non-Alcohol
2,https://patents.google.com/patent/US9040107B2/en,\n A dairy-based yogurt product having a p...,Non-Alcohol
3,https://patents.google.com/patent/US9040036B2/en,\n The present invention relates to pharma...,Non-Alcohol
4,https://patents.google.com/patent/US9038991B2/en,\n A carbonator tank assembly is disclosed...,Non-Alcohol
...,...,...,...
1195,https://patents.google.com/patent/CN103281742B/en,\n The invention discloses a kind of based...,Autonomous Vehicles
1196,https://patents.google.com/patent/CN102806912B/en,\n The present invention relates to the Fa...,Autonomous Vehicles
1197,https://patents.google.com/patent/CN206091477U/en,\n The utility model provides an excessive...,Autonomous Vehicles
1198,https://patents.google.com/patent/EP2544928B1/en,Autonomous Vehicles,NaN


In [4]:
df.dropna(inplace=True)
df

,URL,Abstract,Category
0,/[i9https://patents.google.com/patent/US904030...,\n Methods and compositions for targeted d...,Non-Alcohol
1,https://patents.google.com/patent/US9040109B2/en,\n The present invention provides stabiliz...,Non-Alcohol
2,https://patents.google.com/patent/US9040107B2/en,\n A dairy-based yogurt product having a p...,Non-Alcohol
3,https://patents.google.com/patent/US9040036B2/en,\n The present invention relates to pharma...,Non-Alcohol
4,https://patents.google.com/patent/US9038991B2/en,\n A carbonator tank assembly is disclosed...,Non-Alcohol
...,...,...,...
1193,https://patents.google.com/patent/CN103246284B/en,\n A kind of unmanned control device trave...,Autonomous Vehicles
1194,https://patents.google.com/patent/CN102901510B/en,\n The present invention relates to and in...,Autonomous Vehicles
1195,https://patents.google.com/patent/CN103281742B/en,\n The invention discloses a kind of based...,Autonomous Vehicles
1196,https://patents.google.com/patent/CN102806912B/en,\n The present invention relates to the Fa...,Autonomous Vehicles


In [5]:
corpus = [ df.iloc[i,1].lower() for i in range(len(df))]
corpus

['\n     methods and compositions for targeted delivery of biotherapeutics are provided. the compositions comprise bile-sensitive  st. thermophilus  bacteria modified to release a biotherapeutic agent following bile exposure. biotherapeutic agents released by the  st. thermophilus  bacteria disclosed herein include aq and aqr rich peptides. methods of the invention comprise administering to a subject a  st. thermophilus  bacterium modified to release a biotherapeutic agent following bile exposure. administration of the  st. thermophilus  bacterium promotes a desired therapeutic response. the bacterium may be modified to express and release aq or aqr rich peptides which subsequently inhibit cellular apoptosis or reduce mucosal damage. thus, methods of the invention find use in treating or preventing a variety of gastrointestinal disorders including  c. difficile  infection and antibiotic-associated diarrhea. \n   \n   ',
 '\n     the present invention provides stabilized oil-in-water em

In [6]:
from numpy import array
import numpy as np
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential

Using TensorFlow backend.


AttributeError: module 'keras.backend.tensorflow_backend' has no attribute '_is_tf_1'

In [ ]:
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer
porter=PorterStemmer()

In [ ]:
C:\Users\ASUS\Downloads\glove6b50dtxt\stop_words = set(stopwords.words('english'))

all_words = []
filtered_corpus=[]
for para in corpus:
    filtered_para=''
    sent_list=sent_tokenize(para)
    for sent in sent_list:
        tokenize_word = word_tokenize(sent)
        for word in tokenize_word:
            if word not in stop_words and word not in [',','.']:
                all_words.append(porter.stem(word))
                filtered_para+=porter.stem(word) + ' '
    filtered_corpus.append(filtered_para)
len(all_words)

In [ ]:
filtered_corpus

In [ ]:
unique_words = set(all_words)
vocab_length=len(unique_words)+10
vocab_length

In [ ]:
word_count = lambda para: len(word_tokenize(para))
longest_para = max(corpus, key=word_count)
length_long_para = len(word_tokenize(longest_para))

In [ ]:
tokenizer=Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(filtered_corpus))
embedded_paras=tokenizer.texts_to_sequences(filtered_corpus)
word_index=tokenizer.word_index
word_index.items()

In [ ]:
padded_paras = pad_sequences(embedded_paras, length_long_para, padding='post')

In [ ]:
embedded_paras

In [ ]:
padded_paras.shape

In [ ]:
testy=[]
for x in range(len(df)):
    testy.append([0,0,0,0])
    if df.iloc[x,2]=="Non-Alcohol":
        testy[x][0]=1
    elif df.iloc[x,2]=="Alcohol":
        testy[x][1]=1
    elif df.iloc[x,2]=="Non-Autonomous Vehicles":
        testy[x][2]=1
    elif df.iloc[x,2]=="Autonomous Vehicles":
        testy[x][3]=1
print(testy)

In [ ]:
testy=array([array(xi) for xi in testy])
testy.shape

In [ ]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
import json
EMBEDDING_FILE=r'/home/err_pv/Desktop/Hackathon/text_classification/glove6b50dtxt/glove.6B.50d.txt'
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE, encoding="utf-8_sig"))

In [ ]:
stemmed_keys=[porter.stem(key) for key in embeddings_index.keys()]
embeddings_index = dict(zip(stemmed_keys, list(embeddings_index.values()))) 

In [ ]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()

In [ ]:
embed_size=50
max_features=vocab_length

In [ ]:
embedding_matrix = np.random.normal(emb_mean, emb_std, (max_features, embed_size))

In [ ]:
embedding_matrix

In [ ]:
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix

In [ ]:
model = Sequential()
model.add(Embedding(vocab_length, embed_size, input_length=length_long_para,weights=[embedding_matrix]))
model.add(Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1)))
model.add(GlobalMaxPool1D())
model.add(Dense(50, activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(4, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
len(testy)

In [ ]:
model.fit(padded_paras, testy, epochs=22,validation_split=0.1)

In [ ]:
model.save_weights('model_sub.hdf5')

In [ ]:
df1=pd.read_csv(r'C:\Users\ASUS\Downloads\test_data_live.csv')

In [ ]:
df1

In [ ]:
df1.dropna(inplace=True)

In [ ]:
df1

In [ ]:
corpus_test = [ df1.iloc[i,1] for i in range(len(df1))]

In [ ]:
filtered_corpus_test=[]
for para in corpus_test:
    filtered_para=''
    sent_list=sent_tokenize(para)
    for sent in sent_list:
        tokenize_word = word_tokenize(sent)
        for word in tokenize_word:
            if word not in stop_words and word not in [',','.']:
                filtered_para+=porter.stem(word) + ' '
    filtered_corpus_test.append(filtered_para)
embedded_paras_test=tokenizer.texts_to_sequences(filtered_corpus_test)

In [ ]:
padded_sentences_test = pad_sequences(embedded_paras_test, length_long_para, padding='post')

In [ ]:
predictions = model.predict(padded_sentences_test, batch_size=1024, verbose=1)

In [ ]:
predictions

In [ ]:
result=[]
count=[0,0,0,0]
for x in range(len(predictions)):
    k=max(predictions[x])
    if(predictions[x][0]==k):
        result.append("Non-Alcohol")
        count[0]+=1
    elif(predictions[x][1]==k):
        result.append("Alcohol")
        count[1]+=1
    elif(predictions[x][2]==k):
        result.append("Non-Autonomous Vehicles")
        count[2]+=1
    elif(predictions[x][3]==k):
        result.append("Autonomous Vehicles")
        count[3]+=1
count
    
    
    

In [ ]:
df1=pd.read_csv(r'C:\Users\ASUS\Downloads\test_data_live.csv')

In [ ]:
df1

In [ ]:
final_result=[]

In [ ]:
boo=list(df1["Abstract"].isnull())
boo

In [ ]:
j=0
final_result=[]
for x in range(0,200):
    if not boo[x]:
        final_result.append(result[j])
        j+=1
        #print(x,result[j])
    else:
        final_result.append("Alcohol")
        
len(final_result)
    

In [ ]:
x=159
print(final_result[x],df1.iloc[x][0])

In [ ]:
import csv
f = open('test_data_outputl.csv', 'w')
csvwriter = csv.writer(f)
csvwriter.writerows(final_result)
f.close()

In [ ]:
final_result